# Construction and Test of DNN

In [2]:
import sys
sys.path.insert(0, '../')  # Notebook is in the 'notebooks' directory
from src.dbn import init_DBN, train_DBN, generate_image_DBN
import numpy as np
from matplotlib import pyplot as plt
from src.data_utils import lire_alpha_digit

In [3]:
def init_DNN(sizes, output_size=10):
    configuration = sizes + [output_size]
    return init_DBN(configuration)

In [4]:
def pretrain_DNN(X, dnn, epochs=100, learning_rate=0.1, batch_size=128):
    dbn= {'W': dnn['W'][:-1], 'a': dnn['a'][:-1], 'b': dnn['b'][:-1]}
    dbn = train_DBN(X, dbn, epochs=epochs, learning_rate=learning_rate, batch_size=batch_size)
    dnn['W'][:-1] = dbn['W']
    dnn['a'][:-1] = dbn['a']
    dnn['b'][:-1] = dbn['b']
    return dnn

In [5]:
def calcul_softmax(rbm, X):
    # Calculate the logits
    logits = np.dot(X, rbm['W'][-1]) + rbm['b'][-1]
    
    # Apply the softmax function
    exp_logits = np.exp(logits)
    softmax_probs = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
    
    return softmax_probs

In [ ]:
def entree_sortie_reseau(DNN, X):
    sorties = [X]  # Liste pour stocker les sorties sur chaque couche du réseau
    proba_sortie = None  # Variable pour stocker les probabilités sur les unités de sortie
    
    for i in range(len(DNN['W'])-1):
        RBM = {'W': DNN['W'][i], 'a': DNN['a'][i], 'b': DNN['b'][i]}
        sortie = entree_sortie_RBM(RBM, sorties[-1])
        sorties.append(sortie)
    RBM = {'W': DNN['W'][-1], 'a': DNN['a'][-1], 'b': DNN['b'][-1]}
    proba_sortie = calcul_softmax(RBM, sorties[-1])
    
    return sorties, proba_sortie
